# Плохие результаты

In [1]:
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CyclicLR

# === 1. ЗАГРУЗКА ДАННЫХ ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
        conn
    )

# === 2. КОДИРОВАНИЕ СВЕЧЕЙ (МЕТОД ЛИХОВИДОВА) ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    direction = 1 if close > open_ else 0 if close < open_ else 2
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)

    def classify_shadow(shadow, body):
        if shadow < 0.1 * body:
            return 0  
        elif shadow < 0.5 * body:
            return 1  
        else:
            return 2  

    upper_code = classify_shadow(upper_shadow, body)
    lower_code = classify_shadow(lower_shadow, body)

    return f"{direction}{upper_code}{lower_code}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === 3. ПОДГОТОВКА ДАННЫХ ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

window_size = 20  
predict_offset = 1  

X, y = [], []
for i in range(len(df_fut) - window_size - predict_offset):
    X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    y.append(1 if df_fut['CLOSE'].iloc[i+window_size+predict_offset] > df_fut['CLOSE'].iloc[i+window_size] else 0)

X, y = np.array(X), np.array(y)

split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

class CandlestickDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CandlestickDataset(X_train, y_train)
test_dataset = CandlestickDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# === 4. СОЗДАНИЕ НЕЙРОСЕТИ (LSTM) ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.3):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return self.sigmoid(x)

# === 5. ОБУЧЕНИЕ С РАННЕЙ ОСТАНОВКОЙ, CLR и WEIGHT DECAY ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleLSTM(vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=1).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-2, step_size_up=200, mode="triangular2")

best_accuracy = 0  
model_path = "best_model.pth"
early_stop_epochs = 200  
epochs_no_improve = 0

epochs = 2000
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        scheduler.step()  

        total_loss += loss.item()

    # === Тестирование ===
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).squeeze().round()
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}, Test Accuracy: {accuracy:.2%}")

    # === Ранняя остановка ===
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_path)
        print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_epochs:
        print(f"🛑 Early stopping at epoch {epoch+1}")
        break

# === 6. ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ И ТЕСТ ===
print("\n🔹 Loading best model for final evaluation...")
model.load_state_dict(torch.load(model_path))
model.eval()

correct, total = 0, 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch).squeeze().round()
        correct += (y_pred == y_batch).sum().item()
        total += y_batch.size(0)

final_accuracy = correct / total
print(f"🏆 Final Test Accuracy: {final_accuracy:.2%}")


c:\Users\Alkor\VSCode\quote_prepare\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Epoch 1/2000, Loss: 0.6926, Test Accuracy: 50.50%
✅ Model saved with accuracy: 50.50%
Epoch 2/2000, Loss: 0.6921, Test Accuracy: 49.90%
Epoch 3/2000, Loss: 0.6914, Test Accuracy: 51.49%
✅ Model saved with accuracy: 51.49%
Epoch 4/2000, Loss: 0.6906, Test Accuracy: 50.69%
Epoch 5/2000, Loss: 0.6846, Test Accuracy: 51.49%
Epoch 6/2000, Loss: 0.6794, Test Accuracy: 49.50%
Epoch 7/2000, Loss: 0.6707, Test Accuracy: 50.69%
Epoch 8/2000, Loss: 0.6691, Test Accuracy: 50.69%
Epoch 9/2000, Loss: 0.6650, Test Accuracy: 51.68%
✅ Model saved with accuracy: 51.68%
Epoch 10/2000, Loss: 0.6602, Test Accuracy: 50.89%
Epoch 11/2000, Loss: 0.6491, Test Accuracy: 50.69%
Epoch 12/2000, Loss: 0.6243, Test Accuracy: 52.28%
✅ Model saved with accuracy: 52.28%
Epoch 13/2000, Loss: 0.5998, Test Accuracy: 53.86%
✅ Model saved with accuracy: 53.86%
Epoch 14/2000, Loss: 0.5946, Test Accuracy: 54.46%
✅ Model saved with accuracy: 54.46%
Epoch 15/2000, Loss: 0.5910, Test Accuracy: 55.05%
✅ Model saved with accuracy:

In [2]:
# Загружаем модель
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    prediction = model(last_sequence).item()
    direction = "📈 UP" if prediction > 0.5 else "📉 DOWN"
    print(f"Prediction: {direction} (Probability: {prediction:.2%})")


Prediction: 📉 DOWN (Probability: 25.93%)
